In [4]:
import time
import numpy as np
import pandas as pd
import pylab as plt
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
import enum

In [10]:
class State(enum.IntEnum):
    SUSCEPTIBLE = 0  #agent saint peut être infecté
    INFECTED = 1     # agent infecté
    REMOVED = 2      # agent rétabli





class InfectionModel(Model):
    """A model for infection spread."""

    def __init__(self, N=10, width=10, height=10, ptrans=0.5,
                 death_rate=0.02, recovery_days=21,
                 recovery_sd=7):

        self.num_agents = N
        self.recovery_days = recovery_days
        self.recovery_sd = recovery_sd
        self.ptrans = ptrans
        self.death_rate = death_rate
        self.grid = MultiGrid(width, height, True)
        self.schedule = RandomActivation(self)
        
        #ajout d'un datacolector
        self.datacollector=DataCollector(agent_reporters = {"State" : "state"})
        self.running = True
        self.dead_agents = []
        # Create agents
        for i in range(self.num_agents):
            a = MyAgent(i, self)
            self.schedule.add(a)
            # Add the agent to a random grid cell
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(a, (x, y))
            #make some agents infected at start
            infected = np.random.choice([0,1], p=[0.98,0.02])
            if infected == 1:
                a.state = State.INFECTED
                a.recovery_time = self.get_recovery_time()

        self.datacollector = DataCollector(          
            agent_reporters={"State": "state"})

        
## calcul le nombre de jour de rétablissement (loi normale)
    def get_recovery_time(self):
        return int(self.random.normalvariate(self.recovery_days,self.recovery_sd))

## fonction principale du modele
    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()

In [47]:
class MyAgent(Agent):
    """ An agent in an epidemic model."""
    
    
    # constructeur de l'agent
    
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.age = self.random.normalvariate(20,40)        
        self.state = State.SUSCEPTIBLE  
        self.infection_time = 0

    def move(self):
        """Move the agent"""
                        
        possible_steps = self.model.grid.get_neighborhood(
            self.pos,
            moore=True,
            include_center=False)
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

    def status(self):
        """Check infection status"""

        if self.state == State.INFECTED:     
            drate = self.model.death_rate
            # 2% de chance de décéder 
            alive = np.random.choice([0,1], p=[drate,1-drate])
            
            #si alive = 0 -> l'agent est retirer du systeme avec remove
            if alive == 0:
                self.model.schedule.remove(self)     # l'agent est supprimé du systeme    
            t = self.model.schedule.time-self.infection_time
            #calculer le temps écoulé et vérifier s'il dépasse la durée nécessaire au rétablissement
            if t >= self.recovery_time:         
                self.state = State.REMOVED

    def contact(self):
        """Find close contacts and infect"""
        # récupérer le contenu d'une cellule (vide ou plusieurs agents)

        cellmates = self.model.grid.get_cell_list_contents([self.pos])       
        if len(cellmates) > 1:  #si l'agent n'est pas tout seul
            for other in cellmates:
                if self.random.random() > self.model.ptrans:
                    continue
                if self.state is State.INFECTED and other.state is State.SUSCEPTIBLE:                    
                    other.state = State.INFECTED
                    other.infection_time = self.model.schedule.time
                    other.recovery_time = model.get_recovery_time()

    def step(self):
        self.status()
        self.move()
        self.contact()
        
# programme principal
pop =100
steps=200

model = InfectionModel(pop, 20, 20, ptrans=0.5)
for i in range(steps):
    model.step()

#récupérer le résultat de la collecte des informations concernant les agents   
agent_state = model.datacollector.get_agent_vars_dataframe()


#fonction qui croise les données (affiche le step / états)


def get_column_data(model):
    """pivot the model dataframe to get states count at each step"""
    agent_state = model.datacollector.get_agent_vars_dataframe()
    X = pd.pivot_table(agent_state.reset_index(),index='Step',columns='State',aggfunc=np.size,fill_value=0)    
    labels = ['Susceptible','Infected','Removed']
    X.columns = labels[:len(X.columns)]
    return X

data=get_column_data(model)
data.tail()

,Susceptible,Infected,Removed
Step,,,
195,96,0,2
196,96,0,2
197,96,0,2
198,96,0,2
199,96,0,2
